In [11]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, recall_score, precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve
import torch.nn.functional as F

In [6]:
loaded_datasets_info = torch.load('/root/autodl-tmp/data/saved_datasets.pth', weights_only=False)
train_dataset = loaded_datasets_info['train_dataset']
test_dataset = loaded_datasets_info['test_dataset']

In [7]:
batch_size = 10
loaded_train_dataset = DataLoader(train_dataset, batch_size = batch_size, shuffle = False)
loaded_test_dataset = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
        )
        self.fcs = nn.Sequential(
            nn.Linear(2304, 1152),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1152, 576),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(576, 10)
        )
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

In [9]:
device = "cuda"
model = Net().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(params=model.parameters(), lr=0.001)
num_epochs = 10

In [10]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Print average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / (len(loaded_train_dataset) / batch_size)}")  

Epoch 1/10, Loss: 2.096076072370433
Epoch 2/10, Loss: 0.9259225890847523
Epoch 3/10, Loss: 0.7174603522022901
Epoch 4/10, Loss: 0.6413480141689214
Epoch 5/10, Loss: 0.5525594813361668
Epoch 6/10, Loss: 0.5270170656357491
Epoch 7/10, Loss: 0.5061076240224204
Epoch 8/10, Loss: 0.4707066361641534
Epoch 9/10, Loss: 0.46188682863761005
Epoch 10/10, Loss: 0.43728336004801754


In [ ]:
# get roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC

In [22]:
def calculate_multiclass_metrics(true_labels, predicted_labels, predicted_probabilities, num_classes):
    sensitivity_per_class = []
    specificity_per_class = []
    auc_per_class = []

    accuracy = accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)

    for i in range(num_classes):
        true_binary = (np.array(true_labels) == i).astype(int)
        pred_binary = (np.array(predicted_labels) == i).astype(int)

        cm = confusion_matrix(true_binary, pred_binary, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

        sensitivity_per_class.append(sensitivity)
        specificity_per_class.append(specificity)

        if predicted_probabilities is not None:
            auc = roc_auc_score(true_binary, predicted_probabilities[:, i]) if len(np.unique(true_binary)) > 1 else 0
            auc_per_class.append(auc)

    avg_sensitivity = np.mean(sensitivity_per_class)
    avg_specificity = np.mean(specificity_per_class)
    avg_auc = np.mean(auc_per_class) if auc_per_class else 0

    #print("Classification Report:")
    #print(classification_report(true_labels, predicted_labels, digits=4))
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
    print(f"Average Sensitivity: {avg_sensitivity:.4f}")
    print(f"Average Specificity: {avg_specificity:.4f}")
    print(f"Average AUC: {avg_auc:.4f}")
    
    #for i in range(num_classes):
    #    print(f"Class {i}: Sensitivity = {sensitivity_per_class[i]:.4f}, Specificity = {specificity_per_class[i]:.4f}")
    #    if predicted_probabilities is not None:
    #        print(f"Class {i}: AUC = {auc_per_class[i]:.4f}")

In [23]:
predicted_probabilities = []
true_labels = []
with torch.set_grad_enabled(False):
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)      
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [24]:
true_labels_ = np.argmax(true_labels, axis=-1)
print(true_labels_)

predicted_labels = np.argmax(predicted_probabilities, axis=-1)
print(predicted_labels)

preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)
print(preds)

[5 0 4 ... 5 6 8]
[5 0 4 ... 5 6 8]
tensor([[9.5244e-16, 4.2310e-20, 1.6396e-21,  ..., 2.7411e-20, 1.6484e-14,
         8.5069e-14],
        [1.0000e+00, 1.3299e-24, 2.8630e-16,  ..., 2.8158e-21, 4.5693e-21,
         1.4943e-16],
        [3.7910e-24, 3.2167e-16, 6.9439e-22,  ..., 6.8377e-19, 8.4983e-19,
         5.3311e-11],
        ...,
        [6.7695e-19, 9.9230e-26, 8.8108e-23,  ..., 4.3038e-23, 1.7075e-19,
         1.2504e-17],
        [8.9366e-19, 5.6034e-24, 4.1750e-16,  ..., 5.3479e-30, 2.2247e-21,
         3.0061e-25],
        [2.2347e-14, 2.0734e-17, 1.4224e-11,  ..., 8.6425e-20, 1.0000e+00,
         3.2686e-10]])


In [25]:
calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=10)

Classification Report:
              precision    recall  f1-score   support

           0     0.9629    0.9983    0.9803      5923
           1     0.9882    0.9964    0.9923      6742
           2     0.9888    0.9906    0.9897      5958
           3     0.9982    0.9741    0.9860      6131
           4     0.9872    0.9940    0.9906      5842
           5     0.9836    0.9930    0.9883      5421
           6     0.9946    0.9870    0.9908      5918
           7     0.9928    0.9874    0.9901      6265
           8     0.9932    0.9798    0.9865      5851
           9     0.9900    0.9781    0.9840      5949

    accuracy                         0.9879     60000
   macro avg     0.9879    0.9879    0.9878     60000
weighted avg     0.9880    0.9879    0.9879     60000

Accuracy: 0.9879
Matthews Correlation Coefficient (MCC): 0.9866
Average Sensitivity: 0.9879
Average Specificity: 0.9987
Average AUC: 0.9998
Class 0: Sensitivity = 0.9983, Specificity = 0.9958
Class 0: AUC = 0.9999
Clas

In [26]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_indx, (inputs, labels) in enumerate(loaded_test_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [27]:
true_labels_ = np.argmax(true_labels, axis=-1)
print(true_labels_)

predicted_labels = np.argmax(predicted_probabilities, axis=-1)
print(predicted_labels)

preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)
print(preds)

[7 2 1 ... 4 5 6]
[7 2 1 ... 4 5 6]
tensor([[1.6281e-17, 1.2736e-11, 1.0172e-08,  ..., 1.0000e+00, 7.3936e-10,
         4.9919e-07],
        [2.1103e-21, 4.7740e-16, 1.0000e+00,  ..., 2.9388e-15, 4.6941e-17,
         7.6317e-20],
        [5.5118e-19, 1.0000e+00, 2.4494e-11,  ..., 3.8591e-11, 1.1458e-12,
         3.0847e-11],
        ...,
        [0.0000e+00, 6.2625e-36, 4.6637e-35,  ..., 5.3918e-34, 7.9605e-37,
         2.5768e-24],
        [2.7660e-23, 9.1589e-27, 5.8123e-24,  ..., 1.3825e-18, 9.3278e-20,
         1.0766e-16],
        [7.3203e-17, 7.5009e-16, 6.3968e-21,  ..., 4.1816e-32, 5.5344e-19,
         6.0803e-24]])


In [28]:
calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=10)

Classification Report:
              precision    recall  f1-score   support

           0     0.9597    0.9969    0.9780       980
           1     0.9895    0.9956    0.9925      1135
           2     0.9894    0.9932    0.9913      1032
           3     0.9980    0.9792    0.9885      1010
           4     0.9878    0.9919    0.9898       982
           5     0.9800    0.9910    0.9855       892
           6     0.9915    0.9791    0.9853       958
           7     0.9893    0.9864    0.9878      1028
           8     0.9927    0.9825    0.9876       974
           9     0.9919    0.9732    0.9825      1009

    accuracy                         0.9870     10000
   macro avg     0.9870    0.9869    0.9869     10000
weighted avg     0.9871    0.9870    0.9870     10000

Accuracy: 0.9870
Matthews Correlation Coefficient (MCC): 0.9856
Average Sensitivity: 0.9869
Average Specificity: 0.9986
Average AUC: 0.9997
Class 0: Sensitivity = 0.9969, Specificity = 0.9955
Class 0: AUC = 0.9999
Clas

In [29]:
np.save('/root/autodl-tmp/ROC/CNN/y_val_pred.npy', predicted_probabilities)
np.save('/root/autodl-tmp/ROC/CNN/y_val.npy', true_labels)

FileNotFoundError: [Errno 2] No such file or directory: '/root/autodl-tmp/ROC/CNN/y_val_pred.npy'

In [ ]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_indx, (inputs, labels) in enumerate(loaded_test_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [ ]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities, true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC)

In [ ]:
np.save('/root/autodl-tmp/ROC/CNN/y_test_pred.npy', predicted_probabilities)
np.save('/root/autodl-tmp/ROC/CNN/y_test.npy', true_labels)